# Линейные модели на пальцах

Модуль scipy содержит множество инструментов, предназначенных для научных вычислений. 

Его различные подмодули соответствуют различным областям применения таким как: интерполяция, оптимизация, обработка изображений, статистика, специальные функции и другие.

In [ ]:
import numpy as np

In [ ]:
import scipy
import scipy.optimize

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Попробуем написать самое простое линейное уравнение

Давайте напишем класс, который будеть искать линейную зависимость методом наименьших квадратов

$\sum_{i=1}^n (<w, x_i> - y_i)^2 \to \min_{w}$

заодно узнаем как писать свои классы :)

In [ ]:
class LinearModel(object): # наследуемся от object – стандартная практика
    def __init__(self, loss_function): # конструктор имеет название __init__ и первым аргументом всегда имеет self
        self.loss_function = loss_function
        
    def fit(self, X_data, y_data):
        """
        тут можно написать документацию метода
        X - это выборка признаков, y - выборка целевых переменных
        """
        # пока сделаем реализацию через scipy.optimize
        
        def func(weights):
            return np.sum(self.loss_function(np.dot(X_data, weights[1:]) + weights[0] - y_data))
        
        self.weights = scipy.optimize.minimize(func, np.ones(X_data.shape[1] + 1)).x
        return self
        
    def predict(self, X_data):
        return np.dot(X_data, self.weights[1:]) + self.weights[0]

Генерируем данные

In [ ]:
X_data = np.random.uniform(0, 10, size=50)
y_data = X_data * 0.5 + 7 + np.random.uniform(-1, 1, size=len(X_data))

И визуализируем их

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X_data, y_data)
plt.ylim(0)
plt.xlim(0)
plt.grid()
plt.show()

Далее обучаем модель

In [ ]:
model = LinearModel(lambda arr: np.abs(arr)).fit(X_data[:, np.newaxis], y_data)

И визуализируем её

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X_data, y_data)
plt.plot(X_data, model.predict(X_data[:, np.newaxis]), c='green', linewidth=5)
plt.ylim(0)
plt.xlim(0)
plt.grid()
plt.show()

Объединим всё в одну функцию чтобы было проще рисовать

In [ ]:
def plot_example(error_size=1, loss_function=None, outliers_num=0):
    if loss_function is None:
        loss_function = lambda arr: arr ** 2
    
    X_data = np.random.uniform(0, 10, size=50)
    y_data = X_data * 0.5 + 7 + np.random.uniform(-error_size, error_size, size=len(X_data))
    
    if outliers_num > 0:
        X_out = 1 + np.random.uniform(-0.7, 0.7, size=outliers_num)
        y_out = 20 + np.random.uniform(-0.7, 0.7, size=outliers_num)
        X_data = np.concatenate((X_data, X_out))
        y_data = np.concatenate((y_data, y_out))
    
    model = LinearModel(loss_function).fit(X_data[:, np.newaxis], y_data)
    
    plt.figure(figsize=(10, 5))
    plt.scatter(X_data, y_data)
    plt.plot(X_data, model.predict(X_data[:, np.newaxis]), c='green', linewidth=5)
    plt.ylim(0)
    plt.xlim(0)
    plt.grid()
    plt.show()

In [ ]:
plot_example(1, lambda arr: arr ** 2)

Вроде всё хорошо, но добавим выбросы?

In [ ]:
plot_example(1, lambda arr: arr ** 2, outliers_num=10)

Линия ушла вверх и покосилась, почему так получается? Давайте заменим функцию потрель на модуль? Ну так, чисто по фану

In [ ]:
plot_example(1, lambda arr: np.abs(arr), outliers_num=10)

Внезапно заработало, почему так? Обязательно расскажем на занятии про метрики, а пока можете попробовать сами обосновать

# Попробуем взять функцию посложнее

И использовать seaborn!

In [ ]:
import seaborn as sns
sns.set();

In [ ]:
error_size = 2
X_data = np.random.uniform(-5, 5, size=100)
y_data = 2*X_data**2 - X_data * 0.5 + 7 + np.random.uniform(-error_size, error_size, size=len(X_data))

# ничего не меняется
plt.figure(figsize=(8, 8))
sns.scatterplot(X_data, y_data, s=150);

In [ ]:
error_size = 2
X_data = np.random.uniform(-5, 5, size=100)
y_data = 2*X_data**2 - X_data * 0.5 + 7 + np.random.uniform(-error_size, error_size, size=len(X_data))

model = LinearModel(lambda arr: np.abs(arr)).fit(X_data[:, np.newaxis], y_data)

# ничего не меняется
plt.figure(figsize=(8, 8))
sns.scatterplot(X_data, y_data, s=150);
plt.plot(X_data, model.predict(X_data[:, np.newaxis]), c='green', linewidth=5);

Теперь усложним и модель!

In [ ]:
error_size = 2

X_data = np.sort(np.random.uniform(-5, 5, size=100))
X_col = X_data[:, np.newaxis]
features = np.append(X_col, X_col**2, axis=1)

y_data = 2*X_data**2 - X_data * 0.5 + 7 + np.random.uniform(-error_size, error_size, size=len(X_data))

model = LinearModel(lambda arr: np.abs(arr)).fit(features, y_data)

# ничего не меняется
plt.figure(figsize=(8, 8))
sns.scatterplot(X_data, y_data, s=150);
plt.plot(X_data, model.predict(features), c='green', linewidth=5);